<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#English" data-toc-modified-id="English-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>English</a></span></li><li><span><a href="#Italiano" data-toc-modified-id="Italiano-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Italiano</a></span></li><li><span><a href="#Machiavelli" data-toc-modified-id="Machiavelli-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Machiavelli</a></span></li></ul></div>

In [186]:
import urllib
import pdb
import string
import re
from collections import Counter
import math
import itertools
import pickle
import pandas as pd

# English

In [2]:
url = "https://www-cs-faculty.stanford.edu/~knuth/sgb-words.txt"
file = urllib.request.urlopen(url)
# Build up a list of the words (remove new space character)
word_set = []
for line in file:
    decoded_line = line.decode("utf-8")
    word_set.append(decoded_line.replace('\n',''))

In [ ]:
#put everything in lower letter
word_set = [word.lower() for word in word_set]

In [3]:
l = []
for word in word_set:
    if word[1] == 'o' and word[0] != 'n' and word[2]!= 'r' and word[3] != 'n' and 'r' in word and 'n' in word and 'c' not in word and 'a' not in word and 'e' not in word and 'i' not in word and 's' not in word:
        l.append(word)

In [4]:
def wordle(tries: list, yellow_letter, green_letter):
    wrong = []
    for w in word_set:
        for word in tries:
            wrong.append(word)
            for letter in word:
                if letter not in yellow_letter[tries.index(word)] and letter not in green_letter[tries.index(word)]:
                    if letter in w:
                        wrong.append(w)
                if letter in yellow_letter[tries.index(word)]:
                    if w[word.index(letter)] == letter:
                        wrong.append(w)
                    if letter not in w:
                        wrong.append(w)
                if letter in green_letter[tries.index(word)]:
                    if w[word.index(letter)] != letter:
                        wrong.append(w)
                        
    
    res = [x for x in word_set if x not in wrong]
    return res
                    
                    

In [5]:
wordle(['crane'], [['c','r','a']], [['e']])

['farce']

In [14]:
wordle(['minds', 'lover','block'], [[], ['l','o'],[]], [[], [], ['l','o']])

['float', 'aloft', 'aloof', 'gloat', 'flout', 'aloha', 'zloty']

In [40]:
wordle(['music', 'lover', 'table'], [[], ['l','e'], ['t','a','l','e']], [[],[],[]])

['delta', 'pleat', 'petal', 'fetal']

In [43]:
wordle(['crane', 'fames', 'adept'], [['a', 'e'], ['a', 'e'], ['d']], [[], [], ['a', 'e']])

['ahead']

# Italiano

In [2]:
with open('guesslist_ita.txt', newline='\n') as f:
    to_guess = []
    for line in f:
        to_guess.append(line[:5])
    
with open('wordlist_ita.txt', newline='\n') as f:
    wordlist_ita = []
    for line in f:
        wordlist_ita.append(line[:5])
    

In [68]:
def wordle_game(word_try, word_guess):
    res = [0,0,0,0,0]
    try_count = Counter(word_try)
    guess_count = Counter(word_guess)
    for idx in range(len(word_try)):
        if word_try[idx] not in word_guess:
            res[idx] = 0
            
        if word_try[idx] in word_guess and word_try[idx] != word_guess[idx]:
            res[idx] = 1
            
        if word_try[idx] == word_guess[idx]:
            res[idx] =2
            
    for letter in try_count.keys():
        if try_count[letter] == 2 and guess_count[letter] == 1:
            dup_idx = [index for index, value in enumerate(word_try) if value == letter]
            #pdb.set_trace()
            if res[dup_idx[0]] == 2 and res[dup_idx[-1]] == 1:
                res[dup_idx[-1]] = 0
            if res[dup_idx[0]] == 1 and res[dup_idx[-1]] == 2:
                res[dup_idx[0]] = 0
            if res[dup_idx[0]] == 1 and res[dup_idx[-1]] == 1:
                res[dup_idx[-1]] = 0

        if try_count[letter] == 3 and guess_count[letter] == 1:
            dup_idx = [index for index, value in enumerate(word_try) if value == letter]
            outcome = [res[i] for i in dup_idx]
            if 2 in outcome:
                for i in dup_idx:
                    if res[i] == 1:
                        res[i] = 0
        
        if try_count[letter] == 3 and guess_count[letter] == 2:
            dup_idx = [index for index, value in enumerate(word_try) if value == letter]
            outcome = [res[i] for i in dup_idx]
            if Counter(outcome)[2] == 0:
                res[dup_idx[-1]] = 0
            
            if Counter(outcome)[2] == 1:
                if res[dup_idx[-1]] == 1:
                    res[dup_idx[-1]] = 0
                else:
                    res[dup_idx[-2]] = 0
                    
            if Counter(outcome)[2] == 2:
                for i in dup_idx:
                    if res[i] == 1:
                        res[i] = 0

    return tuple(res)
        
        

In [564]:
def entropy(word_try, word_list): 
    word_dict = {}
    out_dict = {}
    entropy_dict = {}
    prob_dict = {}
    for word in word_try:
        for w in word_list:
            out = wordle_game(word, w)
            #pdb.set_trace()
            if word not in out_dict.keys():
                out_dict[word] = {out:[w]}
            else:
                if out not in out_dict[word].keys():
                    out_dict[word][out] = [w]
                else:
                    out_dict[word][out].append(w)
        
        for p in out_dict[word].keys():
            if word not in prob_dict.keys():
                #pdb.set_trace()
                prob_dict[word] = {p: (len(out_dict[word][p])/len(word_list))}
                entropy_dict[word] = {p: (len(out_dict[word][p])/len(word_list))*math.log2(len(word_list)/len(out_dict[word][p]))}
            else:
                prob_dict[word][p] = (len(out_dict[word][p])/len(word_list))
                entropy_dict[word][p] = (len(out_dict[word][p])/len(word_list))*math.log2(len(word_list)/len(out_dict[word][p]))
                
            word_dict[word] = sum(entropy_dict[word].values())

    return word_dict, out_dict, prob_dict, entropy_dict
        

In [565]:
q,w,e,r = entropy(['salve'], wordlist_ita)

In [645]:
word_dict, out_dict, prob_dict, entropy_dict = entropy(wordlist_ita, wordlist_ita)

In [657]:
entropy_df = pd.DataFrame(word_dict.items(), columns=['word', 'entropy'])
#entropy_df['guess'] = [1 if x in word_list else 0 for x in list(ent_df['word'])]

In [704]:
def recurrent_wordle(word_list, prev_word_list = None, i = 0, out_dict = out_dict, prob_dict = prob_dict, entropy_dict = entropy_dict):
    i += 1
    if i == 1:
        print('The best words are:')
        print(entropy_df.sort_values(by = 'entropy', ascending=False).head())

        
    else: 
        word_dict, out_dict, prob_dict, entropy_dict = entropy(wordlist_ita, word_list)

        ent_df = pd.DataFrame(word_dict.items(), columns=['word', 'entropy'])
        ent_df['guess'] = [1 if x in word_list else 0 for x in list(ent_df['word'])]
        #pdb.set_trace()
        print('The best words are:')
        print(ent_df.sort_values(['entropy', 'guess'], ascending=[False, False]).head())
        
        
        
    print('Please, select new word:')
    new_word = input()
    print('Wordle output')
    new_wordle_outcome = input()
    wordle_outcome = tuple([int(i) for i in new_wordle_outcome])
    
    new_word_list = out_dict[new_word][wordle_outcome]
    final_word_list = [x for x in new_word_list if x in word_list]
    
    print('The probably was:', prob_dict[new_word][wordle_outcome])
    print('Entropy:', entropy_dict[new_word][wordle_outcome])
    print('There are', len(out_dict[new_word][wordle_outcome]), 'possible matches')
    if len(out_dict[new_word][wordle_outcome]) == 1:
        print('the word is:')
        return out_dict[new_word][wordle_outcome][0]
    
    if len(out_dict[new_word][wordle_outcome]) <= 15:
        print(out_dict[new_word][wordle_outcome])
    
    
    
    return recurrent_wordle(final_word_list, word_list, i)


In [649]:
'senta' in wordlist_ita

True

In [744]:
recurrent_wordle(wordlist_ita)

The best words are:
       word   entropy
5956  sarei  6.148073
6164  seria  6.119296
5579  reati  6.065487
6990  tiare  6.047130
5961  sarti  6.044315
Please, select new word:
sarei
Wordle output
00012
The probably was: 0.02387030890988001
Entropy: 0.12862847641342884
There are 187 possible matches
The best words are:
       word   entropy  guess
3637  lento  4.500247      0
1322  celto  4.280221      0
3614  legno  4.264090      0
1333  cento  4.255193      0
3635  lente  4.235270      0
Please, select new word:
lento
Wordle output
01000
The probably was: 0.0855614973262032
Entropy: 0.303477600846001
There are 16 possible matches
The best words are:
       word  entropy  guess
2224  emidi    3.750      1
2136  ecidi    3.625      1
295   amidi    3.625      0
2223  emide    3.625      0
7305  umidi    3.625      0
Please, select new word:
emidi
Wordle output
10112
The probably was: 0.0625
Entropy: 0.25
There are 1 possible matches
the word is:


'dieci'

In [206]:
d = {'aceto':{(1,2,1,1,1):['bello']},
    'isola':{(1,2,1,1,1):['bella']}}

In [227]:
(1,2,1,1,1) in d['aceto'].keys()

True

In [209]:
d['aceto'][(1,2,1,1,1)]

['bello']

In [722]:
entropy_dict = entropy(wordlist_ita, wordlist_ita)

KeyboardInterrupt: 

In [185]:
a_file = open("entropy_dict.pkl", "wb")
pickle.dump(entropy_dict, a_file)
a_file.close()

In [197]:
entropy_df = pd.DataFrame(entropy_dict.items(), columns=['word', 'entropy'])

In [198]:
entropy_df.to_csv('word_entropy.csv')

In [669]:
entropy_df.sort_values('entropy')

,word,entropy
2861,fuzzy,1.803808
3257,hokko,2.173044
3464,kayak,2.179525
7817,zozzo,2.270465
3253,hobby,2.332595
...,...,...
5961,sarti,6.044315
6990,tiare,6.047130
5579,reati,6.065487
6164,seria,6.119296


In [180]:
sorted(entropy_dict.items(), key=lambda kv: kv[1], reverse=True)


[('sarei', 6.148073475406857),
 ('seria', 6.119295941674081),
 ('reati', 6.065486936619879),
 ('tiare', 6.047130479048786),
 ('sarti', 6.044315068036249),
 ('reali', 6.043921429727291),
 ('raito', 6.02970342136936),
 ('serio', 6.024851667806634),
 ('roani', 6.007915833620364),
 ('carie', 6.003899580747038),
 ('siate', 5.999320205754984),
 ('riale', 5.996617891127329),
 ('siero', 5.994557088982317),
 ('soani', 5.989327807310609),
 ('sorai', 5.983339516045124),
 ('resia', 5.97877760375071),
 ('raino', 5.9768128537312295),
 ('cario', 5.970538275384395),
 ('roesi', 5.965144939003495),
 ('tarsi', 5.963251075571574),
 ('ratei', 5.962957756247841),
 ('aresi', 5.9582936907481345),
 ('erasi', 5.954704543601071),
 ('aorte', 5.945196336968339),
 ('salio', 5.944679520154489),
 ('siano', 5.9395014821678895),
 ('sanie', 5.938185390350974),
 ('maori', 5.932936208611774),
 ('ratio', 5.929627371273695),
 ('carsi', 5.9291453559939145),
 ('aiole', 5.924978572817903),
 ('arile', 5.923874519012007),
 ('coi

In [687]:
word_dict['certa']

5.736806906801744

# Machiavelli

In [689]:
ranks = ['A', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']
suits = ['C', 'D', 'H', 'S']
order_cards = [x + y for y in suits for x in ranks ]

True

In [28]:
table = []
hand = []

In [30]:
def play_hand(table, hand):
        
    
    
    
    return x